In [1]:
import src.bsky

bsky_client = src.bsky.init()

text = "\n".join(await src.bsky.get_author_feed_texts(bsky_client, "zkat.tech", 25))

2025-04-02 22:57:49 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text--zkat.tech prefix=bsky.get-author-feed-text- suffix=zkat.tech
cursor: 2025-03-31T23:05:51.421Z
2025-04-02 22:57:49 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-31T23:05:51.421Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-31T23:05:51.421Z suffix=zkat.tech
cursor: 2025-03-30T20:25:49.755Z
2025-04-02 22:57:49 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-30T20:25:49.755Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-30T20:25:49.755Z suffix=zkat.tech
cursor: 2025-03-26T00:46:08.42Z
2025-04-02 22:57:49 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-26T00:46:08.42Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-26T00:46:08.42Z suffix=zkat.tech
cursor: 2025-03-23T06:03:20.358Z
2025-04-02 22:57:49 [info     ] cache              

In [2]:
import nltk
import nltk.corpus

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/kai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import subprocess
import spacy

subprocess.run(["spacy", "download", "en_core_web_md"])
nlp = spacy.load("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 64.1 MB/s eta 0:00:00 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import yake
import nltk
import nltk.corpus
import spacy
import yaml
import pprint

def remove_substring_entries(keyword_list):
    # Sort by length of keyword (longest first), then by score (higher is better)
    keyword_list.sort(key=lambda x: (-x[0], -len(x[1])))

    filtered_keywords = []
    seen_words = set()

    for score, keyword in keyword_list:
        words = set(keyword.split())  # Split phrase into individual words
        if not any(word in seen_words for word in words):
            filtered_keywords.append((score, keyword))
            seen_words.update(words)  # Add words to seen set

    return filtered_keywords

with open("stopwords.yml", "r") as _file:
    custom_stopwords = yaml.load(_file, yaml.Loader)

yake_kw_extractor = yake.KeywordExtractor(
    lan="en",
    top=25,
    dedupLim=1,
    stopwords=set(nltk.corpus.stopwords.words("english") + list(nlp.Defaults.stop_words) + custom_stopwords)
)
keywords = yake_kw_extractor.extract_keywords(text.lower())

keywords = [
    (pair[0], pair[1]) if type(pair[0]) != str else (pair[1], pair[0])
    for pair in keywords
] # this pair comes on in a different order depending on your operating system

keywords = remove_substring_entries(keywords)

print(pprint.pprint(keywords))

keywords = keywords[:5]

print(pprint.pprint(keywords))


In [17]:
import spacy
import json

with open("emojis.json", "r") as _file:
    emojis = json.loads(_file.read())

emoji_scores = []

i = 1
for keyword in keywords:
    print(f"progress: {i} out of {len(keywords)}")
    _keyword = nlp(keyword[1])
    keyword_scores = []
    
    for emoji in emojis:
        _emoji = nlp(emoji["description"])
        keyword_score = keyword[0]
        emoji_score = _keyword.similarity(_emoji) * keyword_score
        keyword_scores.append([_keyword, emoji_score])

    print(keyword_scores[0])
    print(keyword_scores[-1])
    keyword_scores.sort(key = lambda x: -x[1])
    emoji_scores.append(keyword_scores[0])
    i += 1

emoji_scores.sort(key = lambda x: -x[1])

progress: 1 out of 5


/var/folders/37/h0w8431d5dqb2pjq5ws463q00000gn/T/ipykernel_1384/3200305685.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  emoji_score = _keyword.similarity(_emoji) * keyword_score


KeyboardInterrupt: 